<a href="https://colab.research.google.com/github/kellysolow/titanic_gpt2_fine_tuned/blob/main/kill'em_with_the_love_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import (AutoTokenizer, AutoModelForCausalLM, AdamW, get_scheduler,GPT2Config,
                          GPT2ForSequenceClassification, get_cosine_with_hard_restarts_schedule_with_warmup)
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from datetime import date, timedelta

import torch
import os
import math
import pandas as pd
import numpy as np
import re
import datetime as dt
import random
import matplotlib.pyplot as plt
import logging
import datetime as dt

In [ ]:
class Config:
  epochs = 200
  learning_rate = 0.026
  eps = 1e-8
  weight_decay = 0
  batch_size = 1
  cycles = 9


args = Config()

def seed_everything(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  np.random.seed(seed)
  random.seed(seed)

seed_everything(69)

maxlength = 40
pclass = 'seat class'
sibsp = 'with sibling'
parch = 'parents onboard'
cabin = 'cabin number'
embarked = 'embarked port'
path = './drive/MyDrive/kaggle/titanic'
today = date.today()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device : ", device)

using device :  cuda:0


In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

file_handler = logging.FileHandler(f'{path}/log/gpt2.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
df = pd.read_csv(f"{path}/test.csv")
df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
#PClass   94
# SibSp     95
# Parch     96
# Cabin  97
# Embarked 98
# Survived 99



In [ ]:
class titanicDataset(Dataset):
  def __init__(self, use_tokenizer):
    self.dfInput = []
    self.dfOutput = []
    for i in range(len(df['Name'])):
      # txt = (pclass+ str(df["Pclass"][i])+"name" + str(df['Name'][i]) +"sex"+ str(df['Sex'][i])+"age" + str(df['Age'][i]) + sibsp+ str(df["SibSp"][i])+ parch 
      #  +str(df["Parch"][i])+ "ticket" + str(df['Ticket'][i])+"fare"+ str(df['Fare'][i])+cabin+ str(df["Cabin"][i])+embarked+ str(df["Embarked"][i]))
      txt = (pclass+ str(df["Pclass"][i])+"sex"+ str(df['Sex'][i])+"age" + str(df['Age'][i]) + sibsp+ str(df["SibSp"][i])+ parch 
       +str(df["Parch"][i])+ "ticket" + str(df['Ticket'][i])+"fare"+ str(df['Fare'][i])+cabin+ str(df["Cabin"][i])+embarked+ str(df["Embarked"][i]))
      self.dfInput.append(txt)
    # for i in df['Survived']:
    #   self.dfOutput.append(i)

    self.n_examples = len(df['Name'])

    return

  def __len__(self):
    return self.n_examples

  def __getitem__(self,item):
    return {'text': self.dfInput[item]}

    #return {'text': self.dfInput[item], 'label': self.dfOutput[item]}

dataDictionary = titanicDataset(use_tokenizer = tokenizer)

In [ ]:
intent2id = {}
intent2id.update({"died":0})
intent2id.update({"alive":1})


In [ ]:
class Gpt2ClassificationCollator(object):
  def __init__(self, use_tokenizer, labels_encoder, max_sequence_len=None):
    self.use_tokenizer = use_tokenizer
    self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len
    self.labels_encoder = labels_encoder

  def __call__(self, sequences):
    texts = [sequence['text'] for sequence in sequences]
    #labels = [sequence['label'] for sequence in sequences]
    #labels = [self.labels_encoder[label] for label in labels]

    inputs = self.use_tokenizer(text = texts, return_tensors="pt", max_length = self.max_sequence_len, padding = 'max_length', truncation=True).to(device)
    #inputs.update({'labels':torch.tensor(labels)})

    return inputs

In [ ]:
collator = Gpt2ClassificationCollator(use_tokenizer = tokenizer, labels_encoder = intent2id, max_sequence_len = maxlength,)
dataloader = DataLoader (dataDictionary , batch_size = args.batch_size, collate_fn =collator)

In [ ]:
#model_config = GPT2Config.from_pretrained('gpt2', num_labels = 2)
model_path = f'{path}/model/titanic2022-09-22_dict'
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels = 2)

model.load_state_dict(torch.load(model_path))
model.eval()

# model.resize_token_embeddings(len(tokenizer))
# model.config.pad_token_id = model.config.eos_token_id
model.to(device)
print("model loaded")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loaded


In [ ]:
def inference (dataloader, device_):
  global model
  predictions_labels = []


  for batch in tqdm(dataloader, total = len(dataloader)):

    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    with torch.no_grad():

      outputs = model(**batch)
      predictions_labels.append(int(outputs.logits.argmax().cpu().numpy()))



  return predictions_labels



In [ ]:
model_prediction =inference(dataloader, device)

print(model_prediction)


100%|██████████| 418/418 [00:04<00:00, 84.74it/s]

[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
df['Survived'] = 1
for index, i in enumerate(model_prediction):
  df['Survived'][index] = i

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df = df.reset_index(drop = True)
output = pd.DataFrame({'PassengerId': df['PassengerId'], 'Survived': df['Survived']})

#df.to_csv(f"{path}/answer.csv", encoding = 'utf-8-sig', index = False)
output.to_csv(f"{path}/answer.csv", index = False)